In [8]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [9]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/08 12:22:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [14]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

home_sales_df = spark.read.csv("home_sales_revised.csv", header=True, inferSchema=True)


In [15]:
# 2. Create a temporary view of the DataFrame.

home_sales_df.createOrReplaceTempView("home_sales")


In [16]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?

avg_price_4_bedrooms = spark.sql("""
    SELECT year, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms = 4
    GROUP BY year
""")
avg_price_4_bedrooms.show()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `year` cannot be resolved. Did you mean one of the following? [`home_sales`.`id`, `home_sales`.`date`, `home_sales`.`price`, `home_sales`.`view`, `home_sales`.`floors`].; line 2 pos 11;
'Aggregate ['year], ['year, round(avg(price#59), 2) AS avg_price#78]
+- Filter (bedrooms#60 = 4)
   +- SubqueryAlias home_sales
      +- View (`home_sales`, [id#56,date#57,date_built#58,price#59,bedrooms#60,bathrooms#61,sqft_living#62,sqft_lot#63,floors#64,waterfront#65,view#66])
         +- Relation [id#56,date#57,date_built#58,price#59,bedrooms#60,bathrooms#61,sqft_living#62,sqft_lot#63,floors#64,waterfront#65,view#66] csv


In [ ]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?

avg_price_3_bed_3_bath = spark.sql("""
    SELECT year_built, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3
    GROUP BY year_built
""")
avg_price_3_bed_3_bath.show()


In [ ]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?

avg_price_specific = spark.sql("""
    SELECT year_built, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
    GROUP BY year_built
""")
avg_price_specific.show()


In [ ]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating. 
# Although this is a small dataset, determine the run time for this query.

avg_price_view = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE price >= 350000
    GROUP BY view
""")
avg_price_view.show()

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# 7. Cache the the temporary table home_sales.

spark.catalog.cacheTable("home_sales")


In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

In [ ]:
# 9. Using the cached data, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()
avg_price_view_cached = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE price >= 350000
    GROUP BY view
""")
avg_price_view_cached.show()
print("Runtime:", time.time() - start_time)


print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 

home_sales_df.write.partitionBy("date_built").parquet("path/to/home_sales_parquet")


In [ ]:
# 11. Read the formatted parquet data.

parquet_df = spark.read.parquet('parquet_home_sales')


In [ ]:
# 12. Create a temporary table for the parquet data.

parquet_df = spark.read.parquet("path/to/home_sales_parquet")
parquet_df.createOrReplaceTempView("home_sales_parquet")


In [ ]:
# 13. Using the parquet DataFrame, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

avg_price_view_parquet = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales_parquet
    WHERE price >= 350000
    GROUP BY view
""")
avg_price_view_parquet.show()


print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")


In [ ]:
# 15. Check if the home_sales is no longer cached

is_cached_after = spark.catalog.isCached("home_sales")
print(f"Is home